<h1>Binance API.</h1>

<h2>Connect to Binance</h2>

Install Bbinance API wrapper library. Documentation is <a href="https://jkorf.github.io/Binance.Net/">here</a>.

In [ ]:
#r "nuget:Binance.Net, 8.1.2"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages Binance.Net, 8.1.2

Bibance Client Authorization.

To authorization on API it is possible to use `new BinanceClient(new BinanceClientOptions{ ... })`. We are going to work only with public data.

In [ ]:
using Binance.Net.Clients;

var client = new BinanceClient();

Get current BTC/USDT price. 

In [ ]:
var getBtcUsdtPriceData = await client.SpotApi.ExchangeData.GetPriceAsync("BTCUSDT");

decimal price = 0;
if(getBtcUsdtPriceData.Success)
{
    price = getBtcUsdtPriceData.Data.Price;
}

price

28932.18000000

Connection works! We can see last BTC price.

Next step is to take some historical data.

<h2>Simple reading 5 mins candlestick BTC/USDT<h2>

Our candlestick datastructure.

In [ ]:
public class Candlestick
{
    public DateTime Time { get; set; }
    public decimal Open { get; set; }
    public decimal Low { get; set; }
    public decimal High { get; set; }
    public decimal Close { get; set; }
}

Each record in the table is one candle. In our case 5 mins candle.

In [ ]:
using Binance.Net.Enums; // KlineInterval is there

var getBtcUsdtCandlesticks = await client.SpotApi.ExchangeData.GetKlinesAsync("BTCUSDT", KlineInterval.FiveMinutes);

IList<Candlestick> candles;
if (getBtcUsdtCandlesticks.Success)
{
    candles = getBtcUsdtCandlesticks.Data
        .Select(row => new Candlestick
            {
                Time = row.OpenTime,
                Open = row.OpenPrice,
                Low = row.LowPrice,
                High = row.HighPrice,
                Close = row.ClosePrice
            })
        .ToList();
}

Install Data and Plotly packeges.

In [ ]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" 

#r "nuget:Microsoft.ML, 1.7.0"
#r "nuget:Microsoft.ML.AutoML, 0.19.0"
#r "nuget:Microsoft.Data.Analysis, 0.19.0"
#r "nuget:XPlot.Plotly.Interactive, 4.0.6"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.0 Microsoft.ML, 1.7.0 Microsoft.ML.AutoML, 0.19.0 XPlot.Plotly.Interactive, 4.0.6

Rendering Candles.

In [ ]:
using XPlot.Plotly;
using System.Linq;
using Microsoft.AspNetCore.Html;

var chart = Chart.Candlestick(
    candles
    .OrderBy(t => t.Time)
    .Select(row => new Tuple<string, decimal, decimal, decimal, decimal>(
        row.Time.ToString("dd/MM HH:mm"),
        row.Open,
        row.High,
        row.Low,
        row.Close
    ))
);

chart.WithLayout(new Layout.Layout{
    title="BTC/USDT 5min",
    xaxis = new Xaxis { title = "Date" },
    yaxis = new Yaxis { title = "Price (BTC)" }
});

chart.Display();

<!DOCTYPE html>